In [2]:
import os 
import numpy as np
import math
import matplotlib.pyplot as plt
from torchvision.utils import save_image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [37]:
img_shape = (1,28,28)

In [38]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        def block(input,output,normalize = True):
            layers = [nn.Linear(input,output)]
            if normalize:
                layers.append(nn.BatchNorm2d(output,.8))
            layers.append(nn.LeakyReLU(.2,inplace = True))
            return layers
        
        self.model = nn.Sequential(
            *block(100,128,normalize = False),
            *block(128,256),
            *block(256,512),
            *block(512,1024),
            nn.Linear(1024,int(np.prod(img_shape))),
            nn.Tanh()
        )
    
    def forward(self,z):
        img = self.model(z)
        img = img.view(img.size(0),*img_shape)
        return img

In [51]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def block(input,output):
            layers = [nn.Linear(input,output)]
            layers.append(nn.LeakyReLU(.2,inplace = True))
            return layers
        
        self.model = nn.Sequential(
            *block(int(np.prod(img_shape)),512),
            *block(512,256),
            *block(256,1),
            nn.Sigmoid(),
        )
    
    def forward(self,img):
        flatten = img.view(img.size(0),-1)
        checker = self.model(flatten)
        return checker

In [57]:
cuda = True if torch.cuda.is_available() else False
loss = torch.nn.BCELoss()
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    loss.cuda()
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
os.makedirs("images",exist_ok = True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "images",
        train = True,
        download = True,
        transform = transforms.Compose(
            [transforms.Resize(28),transforms.ToTensor(),transforms.Normalize([.5],[.5])]
        ),
    ),
    batch_size = 64,
    shuffle = True,
    )

In [56]:
optimizer_G = torch.optim.AdamW(generator.parameters(),lr = .0002,betas = (.5,.999))
optimizer_D = torch.optim.AdamW(discriminator.parameters(),lr = .0002,betas = (.5,.999))

In [ ]:
for epochs in range():
    for i, (imgs,_) in enumerate(dataloader):

        # Ground truths for adversarial
        valid = Variable(Tensor(imgs.size(0),1).fill)(1.0,requires_grad = False)
        fake = Variable(Tensor(imgs.size(0),1).fill)(0.0,requires_grad = False)

        # Input images into tensor
        real_imgs = 
